In [9]:
import pandas as pd

file_path = '..data/curated/property/property 2.csv'
df = pd.read_csv(file_path)

# Calculate the middle position and divide the data into two copies
mid_index = len(df) // 2

# split data
df_part1 = df.iloc[:mid_index]
df_part2 = df.iloc[mid_index:]

output_path_part1 = '../data/raw/external/API/2/property_addresses_part1.csv'
output_path_part2 = '../data/raw/external/API/2/property_addresses_part2.csv'

df_part1.to_csv(output_path_part1, index=False)
df_part2.to_csv(output_path_part2, index=False)

print(f"file spilt and save into: {output_path_part1} 和 {output_path_part2}")


文件已分割并保存为: /Users/fenglingyi/Downloads/groupwork_generate/property_addresses_part1.csv 和 /Users/fenglingyi/Downloads/groupwork_generate/property_addresses_part2.csv


part 1

In [3]:
import pandas as pd
import geojson
from geojson import Feature, Point, FeatureCollection

houses_df = pd.read_csv('../data/raw/external/API/2/property_addresses_part1.csv') 
supermarkets_df = pd.read_csv('../data/raw/external/API/victoria_supermarkets.csv') 

houses_df = houses_df.dropna(subset=['longitude', 'latitude'])
supermarkets_df = supermarkets_df.dropna(subset=['longitude', 'latitude'])


def create_geojson_feature(row, feature_type='house'):
    point = Point((row['longitude'], row['latitude']))
    properties = {
        'address': row.get('address', 'unknown'),
        'type': feature_type
    }
    return Feature(geometry=point, properties=properties)

house_features = houses_df.apply(lambda row: create_geojson_feature(row, 'house'), axis=1).tolist()
house_feature_collection = FeatureCollection(house_features)

supermarkets_features = supermarkets_df.apply(lambda row: create_geojson_feature(row, 'supermarkets'), axis=1).tolist()
supermarkets_feature_collection = FeatureCollection(supermarkets_features)

houses_geojson_output_path = '../data/raw/external/API/2/property_geojson1.geojson'
supermarkets_geojson_output_path = '../data/raw/external/API/2/vic_supermarket_geojson.geojson'

with open(houses_geojson_output_path, 'w') as f:
    geojson.dump(house_feature_collection, f)

with open(supermarkets_geojson_output_path, 'w') as f:
    geojson.dump(supermarkets_feature_collection, f)

print("Created")

房屋和车站的GeoJSON文件已生成


part 2

In [1]:
import pandas as pd
import geojson
from geojson import Feature, Point, FeatureCollection

houses_df = pd.read_csv('../data/raw/external/API/2/property_addresses_part2.csv')  # 替换为你的房屋CSV文件路径
supermarkets_df = pd.read_csv('../data/raw/external/API/victoria_supermarkets.csv')  # 替换为你的车站CSV文件路径

houses_df = houses_df.dropna(subset=['longitude', 'latitude'])
supermarkets_df = supermarkets_df.dropna(subset=['longitude', 'latitude'])


def create_geojson_feature(row, feature_type='house'):
    point = Point((row['longitude'], row['latitude']))
    properties = {
        'address': row.get('address', 'unknown'), 
        'type': feature_type
    }
    return Feature(geometry=point, properties=properties)

# 转换房屋数据为 GeoJSON Feature
house_features = houses_df.apply(lambda row: create_geojson_feature(row, 'house'), axis=1).tolist()
house_feature_collection = FeatureCollection(house_features)

# 转换车站数据为 GeoJSON Feature
supermarkets_features = supermarkets_df.apply(lambda row: create_geojson_feature(row, 'supermarkets'), axis=1).tolist()
supermarkets_feature_collection = FeatureCollection(supermarkets_features)

houses_geojson_output_path = '../data/raw/external/API/2/property_geojson2.geojson'
supermarkets_geojson_output_path = '../data/raw/external/API/2/vic_supermarket_geojson.geojson'

with open(houses_geojson_output_path, 'w') as f:
    geojson.dump(house_feature_collection, f)

with open(supermarkets_geojson_output_path, 'w') as f:
    geojson.dump(supermarkets_feature_collection, f)

print("Created")

FileNotFoundError: [Errno 2] No such file or directory: '../data/raw/external/API/2/property_addresses_part2.csv'

part 1

In [ ]:

import geojson
import openrouteservice
import pandas as pd
import time

api_key = '5b3ce3597851110001cf624816520a4a150a44c887cfdb6517d286f4' 
client = openrouteservice.Client(key=api_key)

with open('../data/raw/external/API/2/property_geojson1.geojson', 'r') as f:
    rent_geojson = geojson.load(f)

with open('../data/raw/external/API/2/vic_supermarket_geojson.geojson', 'r') as f:
    supermarkets_geojson = geojson.load(f)

def extract_coordinates(features):
    coordinates = []
    for feature in features['features']:
        lon, lat = feature['geometry']['coordinates']
        address = feature['properties'].get('address', 'unknown')
        coordinates.append((lat, lon, address))
    return coordinates

rent_coordinates = extract_coordinates(rent_geojson)
supermarkets_coordinates = extract_coordinates(supermarkets_geojson)


house_coords = [(lon, lat) for lat, lon, _ in rent_coordinates]
supermarkets_coords = [(lon, lat) for lat, lon, _ in supermarkets_coordinates]

def calculate_distance_matrix_in_batches(house_coords, supermarkets_coords, batch_size=2):
    all_shortest_distances = []
    
    for i in range(0, len(house_coords), batch_size):
        batch_houses = house_coords[i:i + batch_size]
        try:
            matrix = client.distance_matrix(
                locations=batch_houses + supermarkets_coords,
                profile='driving-car',
                sources=list(range(len(batch_houses))),
                destinations=list(range(len(batch_houses), len(batch_houses) + len(supermarkets_coords))),
                metrics=['distance'],
                units='km'
            )
            distances = matrix['distances']
            
            
            for j, house_distances in enumerate(distances[:len(batch_houses)]):
                
                valid_distances = [d for d in house_distances if d is not None]
                if valid_distances:
                    min_distance = min(valid_distances)
                    closest_supermarkets_idx = house_distances.index(min_distance)
                    rent_address = rent_coordinates[i + j][2]
                    closest_supermarkets = supermarkets_coordinates[closest_supermarkets_idx][2]
                    all_shortest_distances.append({
                        'rent_address': rent_address,
                        'closest_supermarkets': closest_supermarkets,
                        'minimum_distance_supermarket': min_distance
                    })
        except Exception as e:
            print(f"Error calculating distance matrix: {e}")
        time.sleep(1)  
    
    return all_shortest_distances


shortest_distances = calculate_distance_matrix_in_batches(house_coords, supermarkets_coords, batch_size=2)

shortest_distances_df = pd.DataFrame(shortest_distances)
output_path = '../data/raw/external/API/2/apart_closest_supermarket_distance_final_2.csv'
shortest_distances_df.to_csv(output_path, index=False)

print(f"saved to {output_path}")


结果已保存到 /Users/fenglingyi/Downloads/groupwork_generate/apart_closest_supermarket_distance_final_2.csv


In [ ]:
import geojson
import openrouteservice
import pandas as pd
import time

api_key = '5b3ce3597851110001cf6248dfd6833040c341ebb4797f7491012ac1'
client = openrouteservice.Client(key=api_key)

with open('../data/raw/external/API/2/property_geojson2.geojson', 'r') as f:
    rent_geojson = geojson.load(f)

with open('../data/raw/external/API/2/vic_supermarket_geojson.geojson', 'r') as f:
    supermarkets_geojson = geojson.load(f)


def extract_coordinates(features):
    coordinates = []
    for feature in features['features']:
        lon, lat = feature['geometry']['coordinates']
        address = feature['properties'].get('address', 'unknown')
        coordinates.append((lat, lon, address))
    return coordinates

rent_coordinates = extract_coordinates(rent_geojson)
supermarkets_coordinates = extract_coordinates(supermarkets_geojson)

house_coords = [(lon, lat) for lat, lon, _ in rent_coordinates]
supermarkets_coords = [(lon, lat) for lat, lon, _ in supermarkets_coordinates]

def calculate_distance_matrix_in_batches(house_coords, supermarkets_coords, batch_size=2):
    all_shortest_distances = []
    
    for i in range(0, len(house_coords), batch_size):
        batch_houses = house_coords[i:i + batch_size]
        try:
            matrix = client.distance_matrix(
                locations=batch_houses + supermarkets_coords,
                profile='driving-car',
                sources=list(range(len(batch_houses))),
                destinations=list(range(len(batch_houses), len(batch_houses) + len(supermarkets_coords))),
                metrics=['distance'],
                units='km'
            )
            distances = matrix['distances']
            
            for j, house_distances in enumerate(distances[:len(batch_houses)]):
                valid_distances = [d for d in house_distances if d is not None]
                if valid_distances:
                    min_distance = min(valid_distances)
                    closest_supermarkets_idx = house_distances.index(min_distance)
                    rent_address = rent_coordinates[i + j][2]
                    closest_supermarkets = supermarkets_coordinates[closest_supermarkets_idx][2]
                    all_shortest_distances.append({
                        'rent_address': rent_address,
                        'closest_supermarkets': closest_supermarkets,
                        'minimum_distance_supermarket': min_distance
                    })
        except Exception as e:
            print(f"Error calculating distance matrix: {e}")
        time.sleep(1)  
    
    return all_shortest_distances

shortest_distances = calculate_distance_matrix_in_batches(house_coords, supermarkets_coords, batch_size=2)


shortest_distances_df = pd.DataFrame(shortest_distances)
output_path = '../data/raw/external/API/2/bpart_closest_supermarket_distance_final_2.csv'
shortest_distances_df.to_csv(output_path, index=False)

print(f"saved to {output_path}")


结果已保存到 /Users/fenglingyi/Downloads/groupwork_generate/bpart_closest_supermarket_distance_final_2.csv


In [ ]:
import pandas as pd

df_part1 = pd.read_csv('../data/raw/external/API/2/apart_closest_supermarket_distance_final_2.csv')
df_part2 = pd.read_csv('../data/raw/external/API/2/bpart_closest_supermarket_distance_final_2.csv')

# merge
df_combined = pd.concat([df_part1, df_part2], axis=0)

df_combined.to_csv('../data/curated/external/API/2/closest_supermarket_distance_final_2.csv.csv', index=False)
